![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# AQL Graph Traversal Tutorial

<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/AqlTraversalTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the previous parts of this AQL tutorial series, we have already covered a number of topics:
* [Part 1: CRUD](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb) 
* [Part 2: Limit, Sort, Filter](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlPart2Tutorial.ipynb)
* [Part 3: Join](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlJoinTutorial.ipynb)

In this part we move from a document data model to graphs und learn how AQL combines different data model in one query language.
Please also note that there is an excellen [free Graph Course](https://www.arangodb.com/arangodb-graph-course) which goes into more detail!

Recall, that in addition to the python interface, you can interact with ArangoDB using its web interface to manage collections and execute the queries.

# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time
from IPython.display import JSON

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [ ]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AqlTraversalTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login["dbName"]] 

In [ ]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

Feel free to use to above URL to checkout the UI!

##  Import Data

We will work with the same dataset as in the [first part of this tutorial](https://colab.research.google.com/github/joerg84/ArangoDBUniversity/blob/master/AqlCrudTutorial.ipynb). So let us quickly import that into our temporary database. 

In [ ]:
# Create the Characters Collection 
db.createCollection(name="Characters")

In [ ]:
insert_query = """
LET data = [
    { "name": "Ned", "surname": "Stark", "alive": true, "age": 41, "traits": ["A","H","C","N","P"] },
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

As before let us check the `Characters` collection:

In [ ]:
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

# Graph Traversals

Relations such as between parents and children can be modeled as graph. In ArangoDB, two documents (a parent and a child character document) can be linked by an edge document. Edge documents are stored in edge collections and have two additional attributes: `_from` and `_to`. They reference any two documents by their document IDs (`_id`).

Our characters have the following relations between parents and children (first names only for a better overview):

|Child   | Parent  |
|---|---|
| Robb  | Ned  |
| Sansa  |  Ned |
| Arya  |  Ned |
| Bran  |  Ned |
| Jon  |  Ned |
| Robb  |  Catelyn |
| Sansa  | Catelyn  |
| Arya  |  Catelyn |
| Bran  |  Catelyn |
| Jaime  | Tywin  |
| Cersei  | Tywin  |
| Tyrion  |  Tywin |
| Joffrey  |  Jaime |
| Joffrey  | Cersei  |


We can visualize the same information in a graph, which often is easier to comprehend.

![child_of](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ChildOf_Graph.png?raw=1)

# Creating Edges

To create the required edge documents to store these relations in the database, we can run a query that combines joining and filtering to match up the right character documents, then use their `_id` attribute to insert an edge into an edge collection ChildOf.

In [ ]:
# Create the Characters Collection 
db.createCollection(name="ChildOf", className = 'Edges')

For creating the edges we face one challenge: The character documents don’t have user-defined keys. If they had, it would allow us to create the edges more easily like:

```INSERT { _from: "Characters/robb", _to: "Characters/ned" } INTO ChildOf```


However, creating the edges programmatically based on character names is a good exercise. Breakdown of the query:


In [ ]:
create_edges_query = """
LET data = [
    {
        "parent": { "name": "Ned", "surname": "Stark" },
        "child": { "name": "Robb", "surname": "Stark" }
    }, {
        "parent": { "name": "Ned", "surname": "Stark" },
        "child": { "name": "Sansa", "surname": "Stark" }
    }, {
        "parent": { "name": "Ned", "surname": "Stark" },
        "child": { "name": "Arya", "surname": "Stark" }
    }, {
        "parent": { "name": "Ned", "surname": "Stark" },
        "child": { "name": "Bran", "surname": "Stark" }
    }, {
        "parent": { "name": "Catelyn", "surname": "Stark" },
        "child": { "name": "Robb", "surname": "Stark" }
    }, {
        "parent": { "name": "Catelyn", "surname": "Stark" },
        "child": { "name": "Sansa", "surname": "Stark" }
    }, {
        "parent": { "name": "Catelyn", "surname": "Stark" },
        "child": { "name": "Arya", "surname": "Stark" }
    }, {
        "parent": { "name": "Catelyn", "surname": "Stark" },
        "child": { "name": "Bran", "surname": "Stark" }
    }, {
        "parent": { "name": "Ned", "surname": "Stark" },
        "child": { "name": "Jon", "surname": "Snow" }
    }, {
        "parent": { "name": "Tywin", "surname": "Lannister" },
        "child": { "name": "Jaime", "surname": "Lannister" }
    }, {
        "parent": { "name": "Tywin", "surname": "Lannister" },
        "child": { "name": "Cersei", "surname": "Lannister" }
    }, {
        "parent": { "name": "Tywin", "surname": "Lannister" },
        "child": { "name": "Tyrion", "surname": "Lannister" }
    }, {
        "parent": { "name": "Cersei", "surname": "Lannister" },
        "child": { "name": "Joffrey", "surname": "Baratheon" }
    }, {
        "parent": { "name": "Jaime", "surname": "Lannister" },
        "child": { "name": "Joffrey", "surname": "Baratheon" }
    }
]

FOR rel in data
    LET parentId = FIRST(
        FOR c IN Characters
            FILTER c.name == rel.parent.name
            FILTER c.surname == rel.parent.surname
            LIMIT 1
            RETURN c._id
    )
    LET childId = FIRST(
        FOR c IN Characters
            FILTER c.name == rel.child.name
            FILTER c.surname == rel.child.surname
            LIMIT 1
            RETURN c._id
    )
    FILTER parentId != null AND childId != null
    INSERT { _from: childId, _to: parentId } INTO ChildOf
    RETURN NEW
"""

query_result = db.AQLQuery(create_edges_query)

# Graph Traversal

Now that edges link character documents (vertices), we have a graph we can query to find out who the parents are of another character – or in graph terms, we want to start at a vertex and follow the edges to other vertices in an [AQL graph traversal](https://www.arangodb.com/docs/stable/aql/graphs-traversals.html):

In [ ]:
sansa_parents_query = """
// First find the start node, i.e., sansa
FOR c IN Characters
    FILTER c.name == "Sansa"
    // Then start a Graph traversal from that start node
    FOR v IN 1..1 OUTBOUND c ChildOf
    RETURN v.name
"""

query_result = db.AQLQuery(sansa_parents_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

We can traverse the Graph also in the reverse direction (i.e., `INBOUND`) to find someones children:

In [ ]:
ned_children_query = """
// First find the start node, i.e., ned
FOR c IN Characters
    FILTER c.name == "Ned"
    // Then start a Graph traversal from that start node
    FOR v IN 1..1 INBOUND c ChildOf
    RETURN v.name
"""

query_result = db.AQLQuery(ned_children_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

# Variable Length Traversals and Grandchildren 

You might have wondered about the `IN 1..1 ` specification. This part actually specifies how many hops should be considered for the Traversal.
For the Lannister family, we have relations that span from parent to grandchild. Let’s change the traversal depth to return grandchildren, which means to go exactly two steps:

In [ ]:
tywin_grandchildren_query = """
// First find the start node, i.e., ned
FOR c IN Characters
    FILTER c.name == "Tywin"
    // Then start a Graph traversal from that start node
    FOR v IN 2..2 INBOUND c ChildOf
    RETURN v.name
"""

query_result = db.AQLQuery(tywin_grandchildren_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

It might be a bit unexpected, that Joffrey is returned twice. However, if you look at the graph visualization, you can see that multiple paths lead from Joffrey (bottom right) to Tywin:

![child_of](img/ChildOf_Graph.png)

As a quick fix, change the last line of the query to RETURN DISTINCT v.name to return each value only once. Keep in mind though, that there are [traversal options](https://www.arangodb.com/docs/stable/aql/graphs-traversals.html#syntax) to suppress duplicate vertices early on.

In [ ]:
tywin_grandchildren_query = """
// First find the start node, i.e., ned
FOR c IN Characters
    FILTER c.name == "Tywin"
    // Then start a Graph traversal from that start node
    FOR v IN 2..2 INBOUND c ChildOf
    RETURN DISTINCT v.name
"""

query_result = db.AQLQuery(tywin_grandchildren_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

To return the parents and grandparents of Joffrey, we can walk edges in OUTBOUND direction and adjust the traversal depth to go at least 1 step, and 2 at most (i.e., `IN 1..2`):

In [ ]:
joffrey_ancestors_query = """
FOR c IN Characters
    FILTER c.name == "Joffrey"
    FOR v IN 1..2 OUTBOUND c ChildOf
        RETURN DISTINCT v.name
"""

query_result = db.AQLQuery(joffrey_ancestors_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

With deeper family trees, it is only be a matter of changing the depth values to query for great-grandchildren and similar relations.

# Next Steps

To continue playing and working with ArangoDB beyond the temporary database, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=AQLJoin&utm_medium=Github&utm_campaign=ArangoDB%20University)
* Take the [free Graph Course](https://www.arangodb.com/arangodb-graph-course)  
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at https://www.arangodb.com/arangodb-training-center/

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html